In [1]:
%cd /Users/gwg/dmcm
%matplotlib inline

/Users/gwg/dmcm


In [2]:
import json
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import mygene
from   PIL import Image
import random
import requests
import torch

from   data import GTExV8Config, GTExV8Dataset, loader
from   models import DCGANAE128, DPCCA

In [5]:
cfg = GTExV8Config()

directory = 'experiments/20190131_big_sweep/'\
            'gtexv8_mode-dpcca_batch_size-128_seed-0_latent_dim-10_lr-0.0001_l1_coef-0.5_em_iters-1_clip-1'
cfg.IMG_EMBED_DIM = cfg.GENE_EMBED_DIM = 100
model = DPCCA(cfg, latent_dim=10, use_gene_net=True)
model.load_state_dict(state)

In [6]:
train_loader, test_loader = loader.get_data_loaders(cfg,
                                                    batch_size=128,
                                                    num_workers=1,
                                                    pin_memory=False,
                                                    directory=directory)
dataset = train_loader.dataset

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
all_tissues = dataset.tissues_df.values.squeeze()
le.fit(all_tissues)
tissue_nums = le.transform(all_tissues)

In [9]:
inds = test_loader.sampler.indices
n    = len(inds)

X1 = torch.Tensor(n, cfg.N_CHANNELS, cfg.IMG_SIZE, cfg.IMG_SIZE)
X2 = torch.Tensor(n, cfg.N_GENES)
labels = torch.Tensor(n)
tissues = []

for j, i in enumerate(inds):
    x1, x2 = dataset[i]
    X1[j] = x1
    X2[j] = x2
    labels[j] = float(tissue_nums[i])
    tissues.append(all_tissues[i])

tissues = np.array(tissues)

In [40]:
X1r, X2r = model.forward([X1, X2])

In [44]:
idx = 0
print(tissues[idx])
all_ens = np.array(dataset.genes_df.index.tolist())
mg = mygene.MyGeneInfo()

Liver


In [46]:
x2r_genes = X2r[idx]

inds = np.argsort(x2r_genes.detach().numpy())
x2r_genes = x2r_genes[inds][-100:]
ens = all_ens[inds][-100:]
ens = [e.split('.')[0] for e in ens]

gene_symbols = []
for result in mg.querymany(ens, scopes='ensembl.gene'):
    gene_symbols.append(result['symbol'])
    
print('Predicted genes:\n')
for g, v in zip(gene_symbols, x2r_genes):
    print('%s,%s' % (g, v.item()))

querying 1-100...done.
Finished.
Predicted genes:

AC087289.5,0.6421633362770081
USP3-AS1,0.6421794295310974
GALE,0.6423243880271912
AC096677.1,0.6423442363739014
AC008735.2,0.642562985420227
CD276,0.6427814960479736
TBX6,0.6433507800102234
COL5A1,0.6435887217521667
ZBTB25,0.6436729431152344
MIR100HG,0.6437064409255981
C21orf58,0.6439553499221802
RHOV,0.6439574956893921
AL137003.2,0.644054651260376
EDN2,0.644525408744812
PLXNB2,0.644561231136322
CLDN7,0.6446290016174316
AC020763.4,0.6447361707687378
RPL17P50,0.6447821259498596
PPFIA4,0.6453238725662231
RAB17,0.6454620361328125
HMGN1P3,0.6455253958702087
HCN3,0.6457555294036865
AC010976.1,0.6458199620246887
FOXM1,0.6458280682563782
AL391834.1,0.6463305950164795
AL080317.3,0.6467077136039734
AC024361.1,0.6468437910079956
RAB26,0.6469472646713257
AC007292.1,0.6471239924430847
AC011462.4,0.6473168134689331
HSF4,0.6474409103393555
S100A14,0.6476196646690369
AC245060.5,0.6476927995681763
AC131212.3,0.6479144096374512
RNF208,0.648082494735717

In [48]:
x2_genes = X2[idx]

inds = np.argsort(x2_genes.detach().numpy())
x2_genes = x2_genes[inds][-100:]
ens = all_ens[inds][-100:]
ens = [e.split('.')[0] for e in ens]

gene_symbols = []
for result in mg.querymany(ens, scopes='ensembl.gene'):
    gene_symbols.append(result['symbol'])
    
print('Actual genes:\n')
for g, v in zip(gene_symbols, x2_genes):
    print('%s,%s' % (g, v.item()))

querying 1-100...done.
Finished.
Actual genes:

DLG4,0.8998185992240906
PRR12,0.8998185992240906
FRMD1,0.8998185992240906
LIN7B,0.8998185992240906
IKZF2,0.8998185992240906
KIFC2,0.8998185992240906
SRMS,0.8998185992240906
LOC389602,0.8998185992240906
AC092851.1,0.8998185992240906
AC008703.1,0.8998185992240906
CHIC1,0.8998185992240906
AC130469.1,0.8998185992240906
ANKRD29,0.8998185992240906
ABR,0.8998185992240906
AC010175.1,0.8998185992240906
ADAMTS10,0.9221265912055969
TMSB4XP8,0.9221265912055969
AP000785.1,0.9221265912055969
CNTD2,0.9221265912055969
MCF2,0.9221265912055969
AC117503.2,0.9221265912055969
EVPLL,0.9221265912055969
ZNF720,0.9221265912055969
AL390726.6,0.9221265912055969
GAS2L3,0.9221265912055969
ESPL1,0.9221265912055969
C9orf66,0.9221265912055969
LINC00907,0.9221265912055969
UNC119,0.9221265912055969
DTHD1,0.9221265912055969
R3HDM2,0.9221265912055969
AC027514.2,0.9221265912055969
SRC,0.9221265912055969
LRP5L,0.9221265912055969
AC015813.5,0.9221265912055969
TMSB4XP4,0.922126